# COVID-19 Analysis Platform

In [1]:
from jupyter_dash import JupyterDash

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import dash_bootstrap_components as dbc
import dash_table as dt

Load and preprocess data

In [3]:
df = pd.read_csv("owid-covid-data.csv")

In [4]:
countries = df.location.unique()

Construct the app and callbacks

In [25]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Create server variable with Flask server object for use with gunicorn
server = app.server

dpm_countries = 20

dft = df[df.groupby(['location']).date.transform('max') == df.date]
dft = dft[['location', 'continent', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths']]

app.layout = html.Div([ 
    
    html.H1("CovAP dashboard", className="text-center"),
    
    html.H2("Country wise daily statistics"),
    
    html.Label("Select country:"),
        
    # Country drop down
    html.Div(
         html.Div(dcc.Dropdown(
                id='country-input',
                options=[{'label': i, 'value': i} for i in countries],
                value='Norway'
            ),
        style={'width': '49%', 'display': 'inline-block'}),
    ),

    html.Div([dcc.Graph(id='daily-cases'),
        ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([dcc.Graph(id='daily-deaths')
        ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    
    html.H2("Analysis of top " +str(dpm_countries)+" countries with highest deaths per million"),
    
    html.Div(dcc.RadioItems(
                id='parameter',
                options=[{'label': i, 'value': i} for i in ['Hospital beds per thousand', 'Aged 65 or older(%)']],
                value='Hospital beds per thousand',
#                 labelStyle={'display': 'inline-block'}
            )),
    html.Div(dcc.Graph(id='deaths-per-million')),
    
    html.H2("Latest infection data"),
    html.P("Last updated on: "+df.date.max()),
    
    html.Div(
        dt.DataTable(
        data=dft.to_dict('records'),
        columns=[{'id': c, 'name': c.title().replace('_', ' ')} for c in dft.columns],
        page_size=10,
        sort_action='native',
        filter_action="native",
        sort_by=[{
            "column_id": 'total_cases',
            "direction": "desc"
                }],
        style_cell_conditional=[
        {
            'if': {'column_id': c},
            'textAlign': 'left'
        } for c in ['Date', 'Region']
        ],
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            },
            {
            'if': {
                'filter_query': '{location} = "World"'
            },
            'backgroundColor': 'grey',
            'color': 'white'
            }
        ],
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
    
    ), className="container-fluid")
])


@app.callback([
    dash.dependencies.Output('daily-cases', 'figure'),
    dash.dependencies.Output('daily-deaths', 'figure'),
    dash.dependencies.Output('deaths-per-million', 'figure'),
],
[
    dash.dependencies.Input('country-input', 'value'),
    dash.dependencies.Input('parameter', 'value')
])
def update_graph(country, parameter):
    dff = df[df.location == country]
    test = df[['location', 'date', 'total_deaths_per_million', 'hospital_beds_per_thousand', 'aged_65_older', 'population']]
    test = test[test.groupby(['location']).date.transform('max') == df.date]
    data = test.dropna().reset_index().sort_values("total_deaths_per_million", ascending=False).head(dpm_countries)
   
    figure1 =  {
        'data': [dict(
            x=dff.date,
            y=dff.new_cases,
#             text=country,
#             customdata='Test',
            type='bar',
#             mode='markers',
            marker={
#                 'size': 25,
#                 'opacity': 0.7,
                'color': 'grey',
#                 'line': {'width': 2, 'color': 'purple'}
            }
        )],
        'layout': dict(
            margin={'l': 40, 'b': 30, 't': 60, 'r': 0},
#             height=450,
            hovermode='closest',
            title={
            'text': "Daily new cases",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=20)
            }
        )
    }
    
    figure2 =  {
        'data': [dict(
            x=dff.date,
            y=dff.new_deaths,
#             text=country,
#             customdata='Test',
            type='bar',
            marker={
                'color': 'grey'
            }
        )],
        'layout': dict(
            margin={'l': 40, 'b': 30, 't': 60, 'r': 0},
#             height=450,
            hovermode='closest',
            title={
            'text': "Daily new deaths",
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=20)
            }
        )
    }
    
    y = data.hospital_beds_per_thousand
    if(parameter == "Aged 65 or older(%)"):
        y = data.aged_65_older
    
    figure3 =  {
        'data': [
                dict(
            x=data.location,
            y=y,
#             name=parameter,
#             customdata='Total deaths per million: '+str(data.total_deaths_per_million),
            type='bar',
#             mode='markers',
            marker={
#                 'size': 25,
#                 'opacity': 0.7,
                'color': 'orange',
#                 'line': {'width': 2, 'color': 'purple'}
            },
            hovertext="Total deaths per million: "+data.total_deaths_per_million.round(2).astype(str),
        )],
        'layout': dict(
#             margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
#             height=450,
#             hovermode='closest',
            title= parameter
        )
    }
   
    
    return figure1, figure2, figure3




In [26]:
app.run_server()

Dash app running on http://127.0.0.1:8050/


Serve the app using `run_server`.  Unlike the standard `Dash.run_server` method, the `JupyterDash.run_server` method doesn't block execution of the notebook. It serves the app in a background thread, making it possible to run other notebook calculations while the app is running.

This makes it possible to iterativly update the app without rerunning the potentially expensive data processing steps.

In [ ]:
print(app._terminate_server_for_port(host='127.0.0.1',port='8050'))

In [ ]:
# df[df.total_deaths_per_million.notna()].total_deaths_per_million

dpm =  df[['location', 'date', 'total_deaths_per_million', 'population_density']].groupby(['location']).last().dropna().reset_index().sort_values("total_deaths_per_million", ascending=False).head(10)
top = df[df.location.isin(dpm.location)]
f = top[top.groupby(['location']).date.transform('max') == top.date]
top.groupby(['location']).last()


In [ ]:
f

By default, `run_server` displays a URL that you can click on to open the app in a browser tab. The `mode` argument to `run_server` can be used to change this behavior.  Setting `mode="inline"` will display the app directly in the notebook output cell.

In [ ]:
app.run_server(mode="inline")

When running in JupyterLab, with the `jupyterlab-dash` extension, setting `mode="jupyterlab"` will open the app in a tab in JupyterLab.

```python
app.run_server(mode="jupyterlab")
```